Run Unit Test Cases - pytest
#!pip install pytest

In [5]:
import importlib, inspect, sys

In [6]:
# Find the test cases by name and execute all matching test cases
# Run pytest in a subprocess so Jupyter cell output is readable
import subprocess, sys, shlex
test_name_1 = "test_flight_nl_query_and_search"
test_name_2 = "test_hotels_nl"
test_name_3 = "test_pois_nl"

test_list = [test_name_1, test_name_2, test_name_3]

expr = " or ".join(test_list)

print(f"Running pytest -k \"{expr}\"")
cmd = [sys.executable, "-m", "pytest", "-q", "tests/test_api_wrappers.py", "-k", expr]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print(proc.stdout)

Running pytest -k "test_flight_nl_query_and_search or test_hotels_nl or test_pois_nl"
...                                                                      [100%]
3 passed in 0.03s



Mock client

In [7]:
# Import  mock_client.MockTavilyClient if present.

try:
    mc_mod = importlib.import_module("mock_client")
    MockClient = getattr(mc_mod, "MockTavilyClient")
    print("Imported MockTavilyClient from mock_client.py")
except Exception:
    print("mock_client.py not found or import failed. Using inline fallback MockTavilyClient.")

    

Imported MockTavilyClient from mock_client.py


Load and run MockClient wrapper apis - FlightSearch, HotelSearch, POISearch

In [8]:
import json
from api_wrappers import FlightSearch, HotelSearch, POISearch

client = MockClient()
fs = FlightSearch(client)
hs = HotelSearch(client)
ps = POISearch(client)

# Build and print NL queries
print("Flight NL query:")
try:
    qf = fs.build_nl_query(origin="London", destination="Paris", depart_date="2025-10-15", passengers=1)
    print(qf)
    print("Flight search (parsed):")
    print(json.dumps(fs.search(origin="London", destination="Paris", depart_date="2025-10-15", passengers=1), indent=2))
except Exception as e:
    print("Flight demo failed:", e)

print("\nHotel NL query:")
try:
    qh = hs.build_nl_query(location="Paris", checkin="2025-10-15", checkout="2025-10-17", min_rating=3.5)
    print(qh)
    print("Hotel search (parsed):")
    print(json.dumps(hs.search(location="Paris", checkin="2025-10-15", checkout="2025-10-17"), indent=2))
except Exception as e:
    print("Hotel demo failed:", e)

print("\nPOI NL query:")
try:
    qp = ps.build_nl_query(location="Paris", interests=["museum","park"], radius_km=3.0)
    print(qp)
    print("POI search (parsed):")
    print(json.dumps(ps.search(location="Paris", interests=["museum","park"]), indent=2))
except Exception as e:
    print("POI demo failed:", e)


Flight NL query:
List flights from London to Paris on 2025-10-15, 1 passenger, economy cabin, sort by best price.
Flight search (parsed):
[
  {
    "airline": "AirMock 1",
    "price": 341.33,
    "currency": "USD",
    "segments": [
      {
        "from": "XXX",
        "to": "YYY",
        "duration_mins": 90
      }
    ],
    "raw": {
      "airline": "AirMock 1",
      "price": 341.33,
      "currency": "USD",
      "segments": [
        {
          "from": "XXX",
          "to": "YYY",
          "duration_mins": 90
        }
      ]
    }
  },
  {
    "airline": "AirMock 2",
    "price": 262.49,
    "currency": "USD",
    "segments": [
      {
        "from": "XXX",
        "to": "YYY",
        "duration_mins": 90
      }
    ],
    "raw": {
      "airline": "AirMock 2",
      "price": 262.49,
      "currency": "USD",
      "segments": [
        {
          "from": "XXX",
          "to": "YYY",
          "duration_mins": 90
        }
      ]
    }
  }
]

Hotel NL query:
Search h

Live Test

In [9]:
!pip install requests; dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
from api_wrappers import FlightSearch, HotelSearch, POISearch
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from live_client import LiveClient

client = LiveClient("https://api.tavily.com/search")  # picks up API key from env
payload = {"query": "List flights from London to Paris"}
result = client.request("search", payload)
print(result)

{'query': 'List flights from London to Paris', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.skyscanner.com/routes/lond/pari/london-to-paris.html', 'title': 'Cheap Flights from London (LOND) to Paris (PARI) - Skyscanner', 'content': 'Paris.$20 per passenger.Departing Thu, Oct 9.One-way flight with easyJet.Outbound direct flight with easyJet departing from London Southend on Thu, Oct 9, arriving in Paris Charles de Gaulle.Price includes taxes and charges.From $20, select. Paris.$27 per passenger.Departing Thu, Sep 18.One-way flight with easyJet.Outbound direct flight with easyJet departing from London Southend on Thu, Sep 18, arriving in Paris Charles de Gaulle.Price includes taxes and charges.From $27, select. Paris.$30 per passenger.Departing Thu, Jul 2.One-way flight with easyJet.Outbound direct flight with easyJet departing from London Southend on Thu, Jul 2, arriving in Paris Charles de Gaulle.Price includes taxes and charges.From $30, 

In [14]:
#TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
BASE_URL = "https://api.tavily.com/search"
#HEADERS = {"Authorization": f"Bearer {TAVILY_API_KEY}", "Content-Type": "application/json"}


client = LiveClient(BASE_URL)
#client = MockClient() 
fs = FlightSearch(client)
print("Flight search NL query:", fs.build_nl_query("London", "Paris", "2025-10-15", passengers=1))
res = fs.search(origin="London", destination="Paris", depart_date="2025-10-15", passengers=1)
print("Flights response (parsed):", res)

hs = HotelSearch(client)
print("Hotel search NL query:", hs.build_nl_query("Paris", "2025-10-15", "2025-10-17"))
print("Hotels parsed:", hs.search(location="Paris", checkin="2025-10-15", checkout="2025-10-17"))

ps = POISearch(client)
print("POI query:", ps.build_nl_query("Paris", ["museum", "park"], radius_km=3.0))
print("POIs parsed:", ps.search(location="Paris", interests=["museum", "park"]))


Flight search NL query: List flights from London to Paris on 2025-10-15, 1 passenger, economy cabin, sort by best price.
Flights response (parsed): [{'airline': None, 'price': None, 'currency': 'USD', 'segments': [], 'raw': {'url': 'https://www.google.com/travel/flights/flights-from-london-to-paris.html', 'title': 'Find Cheap Flights from London to Paris (LON - PAR)', 'content': "Prices are for flights from London to Paris for 1 adult in Economy. | Cheapest round-trip flights | $72 | easyJet Nonstop 2 hr Nov 6 — Nov 9 | The cheapest round-trip flight from London to Paris is currently $72 |  | * The cheapest easyJet flight from London to Paris is $72, flying from Nov 6 to Nov 9. * The cheapest Air France flight from London to Paris is $135, flying from Nov 13 to Nov 16. * The cheapest Vueling flight from London to Paris is $82, flying from Nov 14 to Nov 17. What are the cheapest flights from London to Paris? It's easy to use Google Flights to find the cheapest days to fly from London to